# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-09-06 08:43:05--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.8’

     0K .......                                               100%  271K=0.03s

2018-09-06 08:43:05 (271 KB/s) - ‘/tmp/aggregation.csv.8’ saved [8063/8063]

:sh: Scheme missing.
--2018-09-06 08:43:05--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-09-06 08:43:05--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.10’

     0K .                                                     

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 5000000
val dim = 4
val path = "/tmp/aggregation.csv"
val useAggregationDS = false

val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => new RealClusterizable(id.toLong, new RealVector(v)) }

val groundTruePath = "/tmp/labels"

datasetSize: Int = 5000000
dim: Int = 4
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = false
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(0.799261280981757, 0.11461410852882559, 0.11619430893055882, 0.6050202850876077), ArrayBuffer(0.11316109199320834, 0.6656835266429698, 0.8132485862122387, 0.5423850232962416), ArrayBuffer(0.6477991663570888, 0.7082094774261337, 0.2756313931649007, 0.7408394532978514), ArrayBuffer(0.27379711608599866, 0.8135529987470147, 0.06613321423926044, 0.3772438097670565), ArrayBuffer(0.145765044537018, 0.1928032496736065, 0.7305018921812846, 0.48921968688201867), ArrayBuffer(0.6737383275912018, 0.01358402095554323, 0.02417297781629335, 0.6487858176005108), ArrayBuffer(0.69963645...

## Parameters 

In [ ]:
val k = 7
val iterMax = 40
val epsilon = 0.5
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](true)

// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](2)

k: Int = 7
iterMax: Int = 40
epsilon: Double = 0.5
metric1: clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance with root applied
metric2: clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = clustering4ever.math.distances.scalar.Minkowski@5a6693b6


## Run and measure the algorithm time

In [ ]:
val t1 = System.nanoTime

val model = KMeans.run[
              Long,
              mutable.ArrayBuffer[Double],
              mutable.ArrayBuffer[Double],
              RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
              Euclidean[mutable.ArrayBuffer[Double]]
              ](
                parData,
                k,
                epsilon,
                iterMax,
                metric1
              )

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 8169145799157
model: clustering4ever.scala.clustering.kmeans.KMeansModel[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double],clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]],clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]]] = clustering4ever.scala.clustering.kmeans.KMeansModel@3459e482
t2: Long = 8172995715005
res31: Double = 3.849915848


3.849915848

# Compare algorithm between ParArray and ArraySeq container

In [ ]:
val t1 = System.nanoTime

KMeans.run[Long,
    mutable.ArrayBuffer[Double],
    mutable.ArrayBuffer[Double],
    RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
    Euclidean[mutable.ArrayBuffer[Double]]
    ](parData, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime
val seqData = parData.seq
val t3 = System.nanoTime

KMeans.run[Long,
    mutable.ArrayBuffer[Double],
    mutable.ArrayBuffer[Double],
    RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
    Euclidean[mutable.ArrayBuffer[Double]]
    ](seqData, k, epsilon, iterMax, metric1)

val t4 = System.nanoTime



(t4 - t3).toDouble / (t2 - t1)

t1: Long = 7480515356629
t2: Long = 7480547486179
seqData: scala.collection.mutable.Seq[clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(RealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@92eeef7,List(),2147483647), RealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@688598ef,List(),2147483647), RealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@33c52a2,List(),2147483647), RealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@5cd9e0a3,List(),2147483647), RealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@227ee499,List(),2147483647), RealClusterizable(5,clustering4ever.scala.vectorizables.RealVector@6...

0.5037806007242553

## Different ways to apply model to datasets

In [ ]:
val clusterized11 = rawData.map( v => (model.centerPredict(v), v) )
val clusterized12 = model.centerPredict(rawData)

val clusterized21 = parData.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(parData)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((2,ArrayBuffer(15.55, 28.65)), (1,ArrayBuffer(14.9, 27.55)), (1,ArrayBuffer(14.45, 28.35)), (1,ArrayBuffer(14.15, 28.8)), (1,ArrayBuffer(13.75, 28.05)), (1,ArrayBuffer(13.35, 28.45)), (1,ArrayBuffer(13.0, 29.15)), (1,ArrayBuffer(13.45, 27.5)), (1,ArrayBuffer(13.6, 26.5)), (1,ArrayBuffer(12.8, 27.35)), (1,ArrayBuffer(12.4, 27.85)), (1,ArrayBuffer(12.3, 28.4)), (1,ArrayBuffer(12.2, 28.65)), (1,ArrayBuffer(13.4, 25.1)), (1,ArrayBuffer(12.95, 25.95)), (1,ArrayBuffer(12.9, 26.5)), (1,ArrayBuffer(11.85, 27.0)), (1,ArrayBuffer(11.35, 28.0)), (1,ArrayBuffer(11.15, 28.7)), (1,ArrayBuffer(11.25, 27.4)), (1,ArrayBuffer(10.75, 27.7)), (1,ArrayBuffer(10.5, 28.35)), (1,ArrayBu...

## Plot clustering results

In [ ]:
val rawData = clusterized22.map( rc => rc.vector.toArray ).toArray
val labels = clusterized22.map( rc => rc.clusterID ).toArray
plot(rawData, labels, '*', Palette.COLORS)

rawData: Array[Array[Double]] = Array(Array(15.55, 28.65), Array(14.9, 27.55), Array(14.45, 28.35), Array(14.15, 28.8), Array(13.75, 28.05), Array(13.35, 28.45), Array(13.0, 29.15), Array(13.45, 27.5), Array(13.6, 26.5), Array(12.8, 27.35), Array(12.4, 27.85), Array(12.3, 28.4), Array(12.2, 28.65), Array(13.4, 25.1), Array(12.95, 25.95), Array(12.9, 26.5), Array(11.85, 27.0), Array(11.35, 28.0), Array(11.15, 28.7), Array(11.25, 27.4), Array(10.75, 27.7), Array(10.5, 28.35), Array(9.65, 28.45), Array(10.25, 27.25), Array(10.75, 26.55), Array(11.7, 26.35), Array(11.6, 25.9), Array(11.9, 25.05), Array(12.6, 24.05), Array(11.9, 24.5), Array(11.1, 25.2), Array(10.55, 25.15), Array(10.05, 25.95), Array(9.35, 26.6), Array(9.3, 27.25), Array(9.2, 27.8), Array(7.5, 28.25), Array(8.55, 27.45), Ar...

Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 1,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,34,1000x966,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x966,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer

val (_, trueLabelsFrom0) = ExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID).seq

val nmi = ExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

trueLabels: scala.collection.mutable.Buffer[Int] = ArrayBuffer(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,...

0.8451873667264422